In [ ]:
import pandas as pd
import nltk
import unicodedata
import sys
import string
nltk.download('vader_lexicon')
import time
import csv
import matplotlib.pyplot as plt

In [ ]:
df_clothes = pd.read_csv("../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")
df_clothes.head()

CLeaning and Graph Analysis

In [ ]:
df_clothes.dropna(inplace=True)

In [ ]:
ax = df_clothes['Division Name'].value_counts().sort_index().plot(kind='bar', fontsize=14, figsize=(12,10))
ax.set_title('NUmber of Reviews by DIvision Name\n', fontsize=20)
ax.set_xlabel('Division Name', fontsize=18)
ax.set_ylabel('Count', fontsize=18);
plt.show()

In [ ]:
ax = df_clothes['Department Name'].value_counts().sort_index().plot(kind='bar', fontsize=14, figsize=(12,10))
ax.set_title('NUmber of Reviews by Department Name', fontsize=20)
ax.set_xlabel('Department Name', fontsize=18)
ax.set_ylabel('Count', fontsize=18);
plt.show()

In [ ]:
import seaborn as sns
ax = sns.catplot(x='Department Name', y='Rating', kind = 'box', data = df_clothes, height=12, aspect=1)
ax.fig.suptitle('Class Name and Ratings')

Sentiment Analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:

import re

Removing special characters and stop words from review text columns

In [ ]:

df_clothes['Review Text'] = df_clothes['Review Text'].apply(lambda x: (re.sub('@[^\s]+-,','', x)))
df_clothes['Review Text'] = df_clothes['Review Text'].apply(lambda x: re.sub(r'\B#\w*[a-zA-Z]+\w*', '', x))
df_clothes['Review Text'] = df_clothes['Review Text'].str.lower()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download("stopwords")

In [ ]:
nltk.download('punkt')

In [ ]:
stop = stopwords.words('english')
df_clothes["rev_no_stopw"] = df_clothes['Review Text'].apply(lambda x:
                                                            ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
analyzer= SentimentIntensityAnalyzer()
df_clothes['Compound'] = [analyzer.polarity_scores(a)['compound'] for a in df_clothes["rev_no_stopw"]]
df_clothes['Negative'] = [analyzer.polarity_scores(a)['neg'] for a in df_clothes["rev_no_stopw"]]
df_clothes['Positive'] = [analyzer.polarity_scores(a)['pos'] for a in df_clothes["rev_no_stopw"]]
df_clothes['Neutral'] = [analyzer.polarity_scores(a)['neu'] for a in df_clothes["rev_no_stopw"]]

In [ ]:

for a in df_clothes.Compound:
    df_clothes['Analysis'] = df_clothes.Compound.apply(lambda a: 'Positive' if a > 0 else ('Neutral' if a == 0 else 'Negative'))

In [ ]:

df_clothes.head()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
fig, axes = plt.subplots()


plt.hist(df_clothes['Positive'], bins=30, color="green", alpha=0.6)
plt.xlabel("Positive", fontsize=20)
plt.ylabel("count", fontsize=20)
plt.suptitle("Positive Histogram", fontsize=30)

plt.show()

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
text = " ".join(review for review in df_clothes.rev_no_stopw.astype(str))
print ("There are {} words in the combination of all cells in column BLOOM.".format(len(text)))

In [ ]:
text = text

# word cloud image:
wordcloud = WordCloud().generate(text)

# Display image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

****Random Forest to Predict Sentiment of next Review

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:

vectorizer = TfidfVectorizer (max_features=1000, min_df=7, max_df=0.7, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(df_clothes["Review Text"]).toarray()

In [ ]:
labels = df_clothes["Analysis"].values
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.5, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
text_classifier.fit(X_train, y_train)

In [ ]:
y_pred = text_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))